In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="123")

In [3]:
def update():
    query_string = '''
    MATCH (a:ProgrammingLanguage), (b:Field {name: 'SoftwareDevelopment'}) WHERE not (a)-[:UsefulFor]->(b) CREATE (a)-[r:UsefulFor]->(b)
    '''
    conn.query(query_string)

In [4]:
def create_relation(a, b, relation):
    query_string = '''
    MATCH (a {name: "''' + str(a) + '''"}), (b {name: "''' + str(b) + '''"}) WHERE not (a)-[:''' + str(relation) + ''']->(b) CREATE (a)-[r:''' + str(relation) + ''']->(b)
    '''
    conn.query(query_string)

In [5]:
def create_node(name, typ):
    query_string = '''
    CREATE (a:''' + str(typ) + ''' {name: "''' + str(name) + '''"})
    '''
    conn.query(query_string)

In [6]:
def add_tutorial(name, url, tutorialOf):
    query_string = '''
    CREATE (a:Tutorial {name: "''' + str(name) + '''", url: "''' + str(url) + '''"})
    '''
    conn.query(query_string)
    create_relation(name, tutorialOf, "TutorialOf")

In [7]:
def addBook(name, author, bookOn):
    query_string = '''
    CREATE (a:Book {name: "''' + str(name) + '''", author: "''' + str(author) + '''"})
    '''
    conn.query(query_string)
    create_relation(name, bookOn, "BookOn")

In [151]:
add_tutorial(
    "Playing CartPole with the Actor-Critic Method", 
    "https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic", 
    "TensorFlow")

In [213]:
addBook(
    "The Pragmatic Programmer",
    "Andrew Hunt",
    "SoftwareDevelopment"
)

In [207]:
create_node("Operating Systems", "Software")

In [13]:
create_relation("SoftwareDevelopment", "Physics", "BranchOf")

In [8]:
def find_tutorials(name):
    query_string = '''
    MATCH (m)-[:TutorialOf]->(n {name: "''' + name + '''"}) RETURN m.name AS name, m.url AS url
    '''
    tutorials = conn.query(query_string)
    return tutorials

In [9]:
def find_books(name):
    query_string = '''
    MATCH (m)-[:BookOn]->(n {name: "''' + name + '''"}) RETURN m.name AS name, m.author AS author
    '''
    books = conn.query(query_string)
    return books

In [10]:
def write_resources(name):    
    tutorials = find_tutorials(name)
    if len(tutorials) > 0:
        print("Tutorials for " + name + ":")
    for tutorial in tutorials:
        print("\t" + tutorial["name"] + " : " + tutorial["url"])
        
    books = find_books(name)
    if len(books) > 0:
        print("Books on " + name + ":")
    for book in books:
        print("\t" + book["name"] + " by " + book["author"])
    
    if len(books)+len(tutorials)>0:
        print("\n")

In [12]:
def compile_resources(name):
    query_string = '''
    MATCH (n {name: "''' + name + '''"})-[:Requires]->(m) RETURN m.name AS name
    '''
    requires = conn.query(query_string)
    query_string = '''
    MATCH (n {name: "''' + name + '''"})-[:BranchOf]->(m) RETURN m.name AS name
    '''
    branchOf = conn.query(query_string)
    query_string = '''
    MATCH (m)-[:BranchOf]->(n {name: "''' + name + '''"}) RETURN m.name AS name
    '''
    branches = conn.query(query_string)
    query_string = '''
    MATCH (m)-[:UsefulFor]->(n {name: "''' + name + '''"}) RETURN m.name AS name
    '''
    useful = conn.query(query_string)
    query_string = '''
    MATCH (n {name: "''' + name + '''"})-[:UsefulFor]->(m) RETURN m.name AS name
    '''
    usefulFor = conn.query(query_string)
    
    if len(branchOf)==1:
        print(name + " is a branch of " + branchOf[0]["name"])
    elif len(branchOf)>1:
        print(name + " is a branch of " + ', '.join([str(elem["name"]) for elem in branchOf]))
    else:
        print(name + " isn't a branch of anything.")
        
    if len(branches)==1:
        print(branches[0]["name"] + " are its branches")
    elif len(branches)>1:
        print(', '.join([str(elem["name"]) for elem in branches])  + " are its branches")
        
    if len(usefulFor)==1:
        print("It's useful for " + usefulFor[0]["name"])
    elif len(usefulFor)>1:
        print("It's useful for " + ', '.join([str(elem["name"]) for elem in usefulFor]))
        
    if len(useful)==1:
        print("Learning or using it, you might need: " + useful[0]["name"])
    elif len(useful)>1:
        print("Learning or using it, you might need: " + ', '.join([str(elem["name"]) for elem in useful]))
    
    if len(requires)==1:
        print("The following are prerequisites: " + requires[0]["name"])
    elif len(requires)>1:
        print("The following are prerequisites: " + ', '.join([str(elem["name"]) for elem in requires]))
        
    print("\n")
    lst = [name]
    lst += [element["name"] for element in useful]
    lst += [element["name"] for element in branchOf]
    lst += [element["name"] for element in requires]
    for i in lst:
        write_resources(i)

In [13]:
compile_resources('SoftwareDevelopment')

SoftwareDevelopment is a branch of Physics, Engineering
Operating Systems, Compilers, BackEnd, FrontEnd, DevOps are its branches
Learning or using it, you might need: VersionControl, GraphQL, API, Docker, Git, Perl, Ruby, Scala, MATLAB, Dart, PHP, Swift, Rust, Go, C, C++, C#, Java, JavaScript, R, SQL, Python, ComputerScience


Books on SoftwareDevelopment:
	The Pragmatic Programmer by Andrew Hunt


Tutorials for Docker:
	Docker for Beginners : https://docker-curriculum.com/
	Docker for beginners: From Docker Desktop to Deployment : https://www.youtube.com/watch?v=i7ABlHngi1Q
	Deploying Docker Containers : https://docs.aws.amazon.com/AmazonECS/latest/userguide/docker-basics.html


Tutorials for Git:
	LearnGit : https://www.codecademy.com/learn/learn-git
	Lecture 6: Version Control (git) (2020) : https://www.youtube.com/watch?v=2sjqTHE0zok
	LearnGitBranching : https://learngitbranching.js.org/


Books on C:
	The C Programming Language by Kernighan and Ritchie


Tutorials for R:
	Using R 

In [20]:
query_string = '''
MATCH (n:ProgrammingLanguage {name: 'Python'}) RETURN n
'''
conn.query(query_string)

[<Record n=<Node id=0 labels=frozenset({'ProgrammingLanguage'}) properties={'filenameExtension': '.py', 'name': 'Python'}>>]

In [24]:
languages = ['JavaScript', 'Java', 'C#', 'C++', 'C', 'Go', 'R', 'Rust', 'Swift', 'PHP', 'Dart', 'MATLAB', 'Scala', 'Ruby', 'Perl']

for l in languages:
    query_string = '''
    MATCH (n:ProgrammingLanguage {name: "''' + l + '''"}) RETURN n
    '''
    ans = conn.query(query_string)
    if ans == []:
        create_node(l, "ProgrammingLanguage")
        
update()

In [60]:
branches = ['TensorFlow', 'Keras']

for l in branches:
    query_string = '''
    MATCH (n:Library {name: "''' + l + '''"}) RETURN n
    '''
    ans = conn.query(query_string)
    if ans == []:
        create_node(l, "Library")
        create_relation(l, "Python", "LibraryOf") #
        
update()

In [67]:
create_relation("MachineLearning", "ComputerScience", "Requires")

In [61]:
create_node("MachineLearning", "Field")